In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import datetime
from tqdm import tqdm
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
config = f"""
dataset: 
    competition_dir: /kaggle/input/child-mind-institute-detect-sleep-states
    cv_split_path: /kaggle/input/cv_split/train_folds.csv

seed: 46
"""

CFG = yaml.load(config, Loader=yaml.SafeLoader)

In [3]:
labels = pd.read_csv(os.path.join(CFG['dataset']['competition_dir'], 'train_events.csv'))

# 朝と夜の両方がそろってるものだけを残す
check = labels.groupby(["series_id", "night"])["step"].count().reset_index()
check["step"] = check["step"] == 2
check.rename(columns={"step": "safe"}, inplace=True)
labels = labels.merge(check, on=["series_id", "night"], how="left")
labels = labels[labels["safe"] == True]

labels.head()

,series_id,night,event,step,timestamp,safe
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400,True
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400,True
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400,True
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400,True
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400,True


In [4]:
train = pd.read_parquet(CFG["dataset"]["competition_dir"] + "/train_series.parquet")
train.head()

,series_id,step,timestamp,anglez,enmo
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.6367,0.0217
1,038441c925bb,1,2018-08-14T15:30:05-0400,2.6368,0.0215
2,038441c925bb,2,2018-08-14T15:30:10-0400,2.6370,0.0216
3,038441c925bb,3,2018-08-14T15:30:15-0400,2.6368,0.0213
4,038441c925bb,4,2018-08-14T15:30:20-0400,2.6368,0.0215


In [5]:
thresholds = [0, 12, 36, 60, 90, 120, 150, 180, 240, 300, 360]
for series_id, df in tqdm(train.groupby("series_id")):
    df = df.reset_index(drop=True)

    label_df = labels[labels["series_id"] == series_id][["step", "event"]]

    # 目的変数（起床時：1, 就寝時：0）
    df = df.merge(label_df, on="step", how="left")
    df["target"] = 0
    df.loc[df["event"] == "onset", "target"] = -1
    df.loc[df["event"] == "wakeup", "target"] = 1
    df["target"] = df["target"].cumsum() + 1

    # 目的変数（範囲内の閾値の個数）
    if len(label_df) > 0:
        label_dfs = []
        for event, event_df in label_df[["step", "event"]].groupby("event"):
            dfs = []
            for i, thresh in enumerate(thresholds[::-1]):
                for sign in [1, -1]:
                    thresh_df = event_df.copy()
                    thresh_df["step"] = thresh_df["step"] + thresh * sign
                    thresh_df[f"{event}_target"] = i
                    dfs.append(thresh_df)
            event_df = pd.concat(dfs, axis=0).drop_duplicates(subset=["step"], keep="first")
            label_dfs.append(event_df[["step", f"{event}_target"]])
        label_df = pd.merge(label_dfs[0], label_dfs[1], on=["step"], how="outer").sort_values("step").reset_index(drop=True)
        df = df.merge(label_df, on="step", how="left")
        df[["onset_target", "wakeup_target"]] = df[["onset_target", "wakeup_target"]].interpolate(method="linear", limit_direction="both")
    else:
        df["onset_target"] = 0
        df["wakeup_target"] = 0
            
    path = f"/kaggle/input/save_series_csv/csvs/{series_id}.parquet"
    os.makedirs(os.path.dirname(path), exist_ok=True)
    df.to_parquet(path, index=False)

100%|██████████| 277/277 [01:40<00:00,  2.75it/s]
